In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
credit = pd.read_csv('../input/creditcard.csv')
credit['Class'].value_counts().plot(kind = 'bar')
plt.xlabel('Class')
plt.ylabel('Transcation_Num')
plt.title('Transcation_Num by Class')

It is can be seen the data is extremely unbalanced.
Here I use downgrade the data size of class=0
I randomly pick up the same number as class=0 from class =1

In [ ]:
sns.heatmap(credit.corr())

From the chart above, we can tell the V8,V13,V15,V21-V28 does not corelated with Class

In [ ]:
#credit_drop = credit.drop(["V8","V13","V15",'V21','V22','V23','V24','V25','V26',"V27","V28",'Time'],axis =1)

In [ ]:
#get negetive transaction number and index
nege_num = len(credit[credit['Class']==1])
nege_index = np.array((credit[credit['Class']==1]).index)

#get positive transaction index and select the same amount of transcation of negetive
posi_index = np.array(credit[credit['Class']==0].index)
new_posi_index = np.random.choice(posi_index,nege_num,replace=False)

#negetive index add new postive index and get the new credit data
total_index = np.append(nege_index,new_posi_index)
downgrade_credit = credit.iloc[total_index,:]

In [ ]:
#check Class =1 and Class =0 numbers
downgrade_credit['Class'].value_counts()

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import auc,roc_auc_score,recall_score,classification_report,precision_recall_curve

In [ ]:
#get the train set and test set
target = downgrade_credit['Class']
data = downgrade_credit.loc[:,downgrade_credit.columns != 'Class']
x_train,x_test,y_train,y_test = train_test_split(data,target,test_size = 0.33)
print ("There are {0} items in x_train".format(len(x_train)))
print ("There are {0} items in x_text".format(len(x_test)))

In [ ]:
# Do the Logistic Regression

logreg = LogisticRegression(C = 100, penalty = 'l1')
logreg.fit(x_train,y_train.values.ravel())
y_pred= logreg.predict(x_test)
cnf_matrix = confusion_matrix(y_test,y_pred)
plt.matshow(cnf_matrix)
plt.colorbar()

print('cr:', classification_report(y_test,y_pred))
print('recall_score:', recall_score(y_test,y_pred))
print('roc_auc_score:',roc_auc_score(y_test,y_pred))

In [ ]:
y_predprob = logreg.predict_proba(x_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test.values.ravel(),y_predprob)
roc_auc = auc(fpr,tpr)
plt.plot(fpr,tpr)
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('ROC curve')
print (roc_auc)